In [1577]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Analysis run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1579, Finished, Available, Finished)

Libraries imported successfully!
Analysis run on: 2025-08-02 04:02:14


In [1578]:
# Check whether running in Fabric or locally, and set the data location accordingly
if "AZURE_SERVICE" in os.environ:
    is_fabric = True
    data_location = "abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db/Files/forecasting/"
    print("Running in Fabric, setting data location to /lakehouse/default/Files/")
else:
    is_fabric = False
    data_location = ""
    print("Running locally, setting data location to current directory")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1580, Finished, Available, Finished)

Running in Fabric, setting data location to /lakehouse/default/Files/


In [1579]:
# Load the combined sales economic data
data = pd.read_csv(data_location + 'modelGeneratedData/overall_monthly_with_economic_and_future.csv')
# data = pd.read_csv(data_location + 'modelGeneratedData/overall_monthly_with_trend_seasonality_economic_and_future.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {data.shape}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"\nData types:")
print(data.dtypes)
print(f"\nFirst few rows:")
data.head()

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1581, Finished, Available, Finished)

=== DATASET OVERVIEW ===
Dataset shape: (122, 64)
Date range: 2015-01-01 00:00:00 to 2025-02-01 00:00:00

Data types:
Date                                  datetime64[ns]
Quantity Invoiced                            float64
Quantity Invoiced Mean                       float64
Quantity Invoiced Count                      float64
Sales - USD                                  float64
                                           ...      
Gas Price (Lag6)                             float64
Global Supply Chain Pressure Index           float64
GSCPI (Lag1)                                 float64
Manufacturing Orders Volume Index            float64
MOVI (Lag6)                                  float64
Length: 64, dtype: object

First few rows:


,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Sales - USD,Sales - USD Mean,Sales - USD Count,Unique Customers,Unique Products,Unique Categories,...,data_Factory_Utilization,data_Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6)
0,2015-01-01,22725114.0,68039.263473,334.0,1422359.94,4271.351171,333.0,160.0,53.0,4.0,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.50,-0.39,86.8,76.1
1,2015-02-01,23032809.0,63276.947802,364.0,1433609.73,3949.338099,363.0,183.0,52.0,4.0,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.32,-0.50,98.8,89.3
2,2015-03-01,27527951.0,76679.529248,359.0,1555441.45,4344.808520,358.0,180.0,50.0,4.0,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.38,-0.32,90.0,91.5
3,2015-04-01,25864804.0,68789.372340,376.0,1485847.04,3951.720851,376.0,177.0,58.0,4.0,...,0.4372,76.8014,0.2276,0.2302,14.46,15.23,-0.35,-0.38,83.3,87.4
4,2015-05-01,22517479.0,77379.652921,291.0,1344651.08,4620.794089,291.0,146.0,51.0,4.0,...,0.4381,76.8657,0.2276,0.2302,14.46,15.23,-0.54,-0.35,98.0,88.1


### Feature Engineering - Adding the Best Features

1. "/lakehouse/default/Files/Orders by Created Date Past 10 Years.csv": 'Net Order Value USD'

In [1580]:
# # Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# # Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
# notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
# import pandas as pd
# # Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/Orders by Created Date Past 10 Years.csv')
# orders_df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/Orders by Created Date Past 10 Years.csv'))
# display(orders_df)

# # 1: Convert the 'Date' column to datetime format to ensure compatibility
# orders_df['Attribute Full Date'] = pd.to_datetime(orders_df['Attribute Full Date'])
# orders_df['Net Order Value USD'] = pd.to_numeric(orders_df['Net Order Value USD'], errors='coerce')

# # 2: Align dates to month-start
# orders_df['Date'] = orders_df['Attribute Full Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # 3: Aggregate monthly total order value
# monthly_orders = orders_df.groupby('Date', as_index=False)['Net Order Value USD'].sum()

# # 4: Merge into main data
# data = pd.merge(data, monthly_orders, on='Date', how='left')

# # 5: Fill missing values
# data['Net Order Value USD'] = data['Net Order Value USD'].fillna(method='ffill').fillna(method='bfill')

# # Final check
# print(data.isnull().sum())
# display(data.head())



StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1582, Finished, Available, Finished)

2. "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/CPI_Monthly_2020Base_Germany.csv": 'CPI'

In [1581]:
# # Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# # Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
# notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
# import pandas as pd
# # Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/CPI_Monthly_2020Base_Germany.csv')
# df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/CPI_Monthly_2020Base_Germany.csv'))
# display(df)

# # 1: Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])
# df['CPI'] = pd.to_numeric(df['CPI'], errors='coerce')

# # 2: Align dates to month-start
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # 3: Merge into main data
# data = pd.merge(data, df[['Date', 'CPI']], on='Date', how='left')

# # 4: Fill missing values
# data['CPI'] = data['CPI'].fillna(method='ffill').fillna(method='bfill')

# # 5: Final check
# print(data.isnull().sum())
# display(data.head())

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1583, Finished, Available, Finished)

3. "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/hicp_data.csv": 'HICP'

In [1582]:
# # Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# # Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
# notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
# import pandas as pd
# # Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/hicp_data.csv')
# df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/hicp_data.csv'))
# display(df)

# # 1: Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])
# df['HICP'] = pd.to_numeric(df['HICP'], errors='coerce')

# # 2: Align dates to month-start
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # 3: Merge into main data
# data = pd.merge(data, df[['Date', 'HICP']], on='Date', how='left')

# # 4: Fill missing values
# data['HICP'] = data['HICP'].fillna(method='ffill').fillna(method='bfill')

# # 5: Final check
# print(data.isnull().sum())
# display(data.head())


StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1584, Finished, Available, Finished)

4. "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/new_manufacturing_orders_v2.csv": 'volume_index'


In [1583]:
# # Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# # Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
# notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
# import pandas as pd
# # Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/new_manufacturing_orders_v2.csv')
# df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/new_manufacturing_orders_v2.csv'))
# display(df)

# # 1: Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])
# df['volume_index'] = pd.to_numeric(df['volume_index'], errors='coerce')

# # 2: Align dates to month-start
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # 3: Merge into main data
# data = pd.merge(data, df[['Date', 'volume_index']], on='Date', how='left')

# # 4: Fill missing values
# data['volume_index'] = data['volume_index'].fillna(method='ffill').fillna(method='bfill')

# # 5: Final check
# print(data.isnull().sum())
# display(data.head())

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1585, Finished, Available, Finished)

5. "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/Crude Oil.csv": 'Crude Oil Brent Europe Price in EUR'

In [1584]:
# # Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# # Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
# notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
# import pandas as pd
# # Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/Crude Oil.csv')
# df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/Crude Oil.csv'))
# display(df)

# # 1: Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])
# df['Crude Oil Brent Europe Price in EUR'] = pd.to_numeric(df['Crude Oil Brent Europe Price in EUR'], errors='coerce')

# # 2: Align dates to month-start
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # 3: Merge into main data
# data = pd.merge(data, df[['Date', 'Crude Oil Brent Europe Price in EUR']], on='Date', how='left')

# # 4: Fill missing values
# data['Crude Oil Brent Europe Price in EUR'] = data['Crude Oil Brent Europe Price in EUR'].fillna(method='ffill').fillna(method='bfill')

# # 5: Final check
# print(data.isnull().sum())
# display(data.head())

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1586, Finished, Available, Finished)

6. "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/Global Supply Chain Pressure Index (GSCPI) - updated.csv": 'Global Supply Chain Pressure Index'

In [1585]:
# # Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# # Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
# notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
# import pandas as pd
# # Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/Global Supply Chain Pressure Index (GSCPI) - updated.csv')
# df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/Global Supply Chain Pressure Index (GSCPI) - updated.csv'))
# display(df)

# # 1: Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])
# df['Global Supply Chain Pressure Index'] = pd.to_numeric(df['Global Supply Chain Pressure Index'], errors='coerce')

# # 2: Align dates to month-start
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # Drop the old column from data
# if 'Global Supply Chain Pressure Index' in data.columns:
#     data = data.drop(columns=['Global Supply Chain Pressure Index'])

# # 3: Merge into main data
# data = pd.merge(data, df[['Date', 'Global Supply Chain Pressure Index']], on='Date', how='left')

# # 4: Fill missing values
# data['Global Supply Chain Pressure Index'] = data['Global Supply Chain Pressure Index'].fillna(method='ffill').fillna(method='bfill')

# # 5: Final check
# print(data.isnull().sum())
# display(data.head())

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1587, Finished, Available, Finished)

7. "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/USD to EUR historical exchange rates.csv": 'USD to EUR Exchange Rate'

In [1586]:
# Mount external lakehouse  "DKODEDataLakehouse" into specified local path.
# Note: NotebookUtils is only supported on runtime v1.2 and above. If you are using runtime v1.1, please use mssparkutils instead.
notebookutils.fs.mount("abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db", "/lakehouse/DKODEDataLakehouse")
import pandas as pd
# Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/USD to EUR historical exchange rates.csv')
df = pd.read_csv(notebookutils.fs.getMountPath('/lakehouse/DKODEDataLakehouse/Files/forecasting/userProvidedData/Germany Economic data/USD to EUR historical exchange rates.csv'))
display(df)

# 1: Convert the 'Date' column to datetime format to ensure compatibility
df['Date'] = pd.to_datetime(df['Date'])
df['USD to EUR Exchange Rate'] = pd.to_numeric(df['USD to EUR Exchange Rate'], errors='coerce')

# 2: Align dates to month-start
df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# 3: Merge into main data
data = pd.merge(data, df[['Date', 'USD to EUR Exchange Rate']], on='Date', how='left')

# 4: Fill missing values
data['USD to EUR Exchange Rate'] = data['USD to EUR Exchange Rate'].fillna(method='ffill').fillna(method='bfill')

# 5: Final check
print(data.isnull().sum())
display(data.head())


StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1588, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 878b0010-74be-427a-869b-4f832de3f853)

SynapseWidget(Synapse.DataFrame, 5e6635ef-df46-4cfb-8bd4-fd844542f104)

Date                                  0
Quantity Invoiced                     0
Quantity Invoiced Mean                0
Quantity Invoiced Count               0
Sales - USD                           0
                                     ..
Global Supply Chain Pressure Index    0
GSCPI (Lag1)                          0
Manufacturing Orders Volume Index     0
MOVI (Lag6)                           0
USD to EUR Exchange Rate              0
Length: 65, dtype: int64


SynapseWidget(Synapse.DataFrame, 4602fccf-9720-4f7a-9b3f-24fc3a02a77a)

## Data Preparation for Overall Forecasting

We'll prepare the data for overall quantity forecasting by aggregating all sales data across segments and subcategories to create a single time series for the total quantity.

In [1587]:
# Create overall aggregation
print("=== CREATING OVERALL AGGREGATION ===")

# Print column names
print(f"Columns in dataset: {data.columns.tolist()}")

# Rename columns in the data
data = data.rename(columns={
    'data_PP_Spot': 'PP_Spot',
    'data_Resin': 'Resin',
    'data_WTI_Crude_Oil': 'WTI_Crude_Oil',
    'data_Natural_Gas': 'Natural_Gas',
    'data_Energy_Average': 'Energy_Average',
    'data_PPI_Freight': 'PPI_Freight',
    'data_PMI_Data': 'PMI_Data',
    'data_Factory_Utilization': 'Factory_Utilization',
    'data_Capacity_Utilization': 'Capacity_Utilization',
    'data_Beverage': 'Beverage',
    'data_Household_consumption': 'Household_consumption',
    'data_packaging': 'packaging',
    'data_Diesel': 'Diesel',
    'data_PPI_Delivery': 'PPI_Delivery',
    'data_Oil-to-resin': 'Oil-to-resin',
    'Electricity Price': 'Electricity Price',
    'Electricity Price (Lag6)': 'Electricity Price (Lag6)',
    'Gas Price': 'Gas Price',
    'Gas Price (Lag6)': 'Gas Price (Lag6)',
    'Global Supply Chain Pressure Index': 'Global Supply Chain Pressure Index',
    'GSCPI (Lag1)': 'GSCPI (Lag1)',
    'Manufacturing Orders Volume Index': 'Manufacturing Orders Volume Index',
    'MOVI (Lag6)': 'MOVI (Lag6)',
    'packaging (Lag2)': 'packaging (Lag2)',
    'PPI_Freight (Lag2)': 'PPI_Freight (Lag2)',
    # 'trend': 'decomp_trend',
    'seasonality': 'decomp_seasonality',
    # 'Net Order Value USD': 'Net Order Value USD', #1
    # 'CPI': 'CPI', #2
    # 'HICP': 'HICP', #3
    # 'volume_index': 'Volume Index', #4
    # 'Crude Oil Brent Europe Price in EUR': 'Crude Oil Brent Europe Price in EUR', #5
    # # 'Global Supply Chain Pressure Index': 'Global Supply Chain Pressure Index', #6
    'USD to EUR Exchange Rate': 'USD to EUR Exchange Rate' #7
})

# Define exogenous variables for modeling
exog_vars = [
    'PP_Spot',
    'Resin',
    'PMI_Data',
    'Natural_Gas',
    # 'WTI_Crude_Oil',
    #'Factory_Utilization',
    'packaging',
    'Energy_Average',
    'Electricity Price (Lag6)',
    'Gas Price (Lag6)',
    'Global Supply Chain Pressure Index', # neutral
    # 'future_orders_qty_total',
    'future_orders_qty_next_1m',
    'future_orders_qty_next_3m',
    #'future_orders_qty_next_6m',
    # 'future_orders_qty_next_12m',
    # 'future_orders_avg_lead_time',
    # 'future_orders_min_lead_time',
    # 'future_orders_max_lead_time',
    # 'decomp_trend', 
    # 'decomp_seasonality'
    # 'Net Order Value USD',  #1
    # 'CPI', #2
    # 'HICP', #3
    # 'Volume Index', #4
    # 'Crude Oil Brent Europe Price in EUR', #5
    # # 'Global Supply Chain Pressure Index', #6
    'USD to EUR Exchange Rate' #7
]

# Overall aggregation (sum across all segments and subcategories)
print(f"Overall time series shape: {data.shape}")

# Display summary statistics
print("\n=== OVERALL SUMMARY ===")
print(f"Overall total quantity range: {data['Quantity Invoiced'].min():,.0f} - {data['Quantity Invoiced'].max():,.0f}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"Number of data points: {len(data)}")

# Display columns and their data types
print("\n=== COLUMNS AND DATA TYPES ===")
print(data.dtypes)

data.head()

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1589, Finished, Available, Finished)

=== CREATING OVERALL AGGREGATION ===
Columns in dataset: ['Date', 'Quantity Invoiced', 'Quantity Invoiced Mean', 'Quantity Invoiced Count', 'Sales - USD', 'Sales - USD Mean', 'Sales - USD Count', 'Unique Customers', 'Unique Products', 'Unique Categories', 'Unique SubCategories', 'Unique EndMarkets L1', 'Unique EndMarkets L2', 'future_orders_count', 'future_orders_qty_total', 'future_orders_qty_next_1m', 'future_orders_qty_next_3m', 'future_orders_qty_next_6m', 'future_orders_qty_next_12m', 'future_orders_sales_next_1m', 'future_orders_sales_next_3m', 'future_orders_sales_next_6m', 'future_orders_sales_next_12m', 'future_orders_avg_lead_time', 'future_orders_min_lead_time', 'future_orders_max_lead_time', 'future_orders_due_next_month', 'future_orders_due_next_quarter', 'future_orders_unique_customers', 'future_orders_unique_products', 'future_to_current_ratio', 'qty_rolling_avg_3m', 'future_to_rolling_3m_ratio', 'qty_rolling_avg_12m', 'future_to_rolling_12m_ratio', 'has_future_orders', 

,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Sales - USD,Sales - USD Mean,Sales - USD Count,Unique Customers,Unique Products,Unique Categories,...,Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6),USD to EUR Exchange Rate
0,2015-01-01,22725114.0,68039.263473,334.0,1422359.94,4271.351171,333.0,160.0,53.0,4.0,...,76.7556,0.2276,0.2302,14.46,15.23,-0.50,-0.39,86.8,76.1,0.861615
1,2015-02-01,23032809.0,63276.947802,364.0,1433609.73,3949.338099,363.0,183.0,52.0,4.0,...,76.7556,0.2276,0.2302,14.46,15.23,-0.32,-0.50,98.8,89.3,0.881076
2,2015-03-01,27527951.0,76679.529248,359.0,1555441.45,4344.808520,358.0,180.0,50.0,4.0,...,76.7556,0.2276,0.2302,14.46,15.23,-0.38,-0.32,90.0,91.5,0.923383
3,2015-04-01,25864804.0,68789.372340,376.0,1485847.04,3951.720851,376.0,177.0,58.0,4.0,...,76.8014,0.2276,0.2302,14.46,15.23,-0.35,-0.38,83.3,87.4,0.925450
4,2015-05-01,22517479.0,77379.652921,291.0,1344651.08,4620.794089,291.0,146.0,51.0,4.0,...,76.8657,0.2276,0.2302,14.46,15.23,-0.54,-0.35,98.0,88.1,0.895522


In [1588]:
# Visualize the overall time series
fig = go.Figure()

# Plot Overall Total Quantity
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Quantity Invoiced'],
              mode='lines+markers', name='Overall Total Quantity',
              line=dict(color='darkblue', width=3),
              marker=dict(size=6))
)

fig.update_layout(
    height=600,
    title_text="Overall Sales Quantity Time Series Analysis",
    xaxis_title="Date",
    yaxis_title="Quantity Invoiced",
    showlegend=True,
    template="plotly_white"
)

fig.show()

# Statistical summary
print("\n=== STATISTICAL SUMMARY ===")
print("Overall Quantity Statistics:")
print(data['Quantity Invoiced'].describe())

# Additional time series analysis
print(f"\nTime Series Characteristics:")
print(f"Mean: {data['Quantity Invoiced'].mean():,.0f}")
print(f"Standard Deviation: {data['Quantity Invoiced'].std():,.0f}")
print(f"Coefficient of Variation: {(data['Quantity Invoiced'].std() / data['Quantity Invoiced'].mean() * 100):.2f}%")

# Check for trends and seasonality
data_monthly = data.set_index('Date')['Quantity Invoiced']
monthly_avg = data_monthly.groupby(data_monthly.index.month).mean()
print(f"\nMonthly Averages:")
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month, avg in monthly_avg.items():
    print(f"  {month_names[month-1]}: {avg:,.0f}")

# Year-over-year growth
yearly_avg = data_monthly.groupby(data_monthly.index.year).mean()
print(f"\nYearly Averages:")
for year, avg in yearly_avg.items():
    print(f"  {year}: {avg:,.0f}")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1590, Finished, Available, Finished)


=== STATISTICAL SUMMARY ===
Overall Quantity Statistics:
count    1.220000e+02
mean     2.571784e+07
std      3.955959e+06
min      1.256533e+07
25%      2.309910e+07
50%      2.590907e+07
75%      2.824755e+07
max      3.352502e+07
Name: Quantity Invoiced, dtype: float64

Time Series Characteristics:
Mean: 25,717,842
Standard Deviation: 3,955,959
Coefficient of Variation: 15.38%

Monthly Averages:
  Jan: 26,139,639
  Feb: 25,786,211
  Mar: 28,644,908
  Apr: 24,628,411
  May: 25,084,145
  Jun: 27,862,406
  Jul: 27,817,574
  Aug: 26,186,554
  Sep: 28,143,352
  Oct: 25,329,172
  Nov: 24,806,704
  Dec: 18,136,016

Yearly Averages:
  2015: 24,146,230
  2016: 24,500,382
  2017: 26,040,790
  2018: 25,449,790
  2019: 28,086,842
  2020: 27,556,023
  2021: 28,535,274
  2022: 26,028,150
  2023: 22,195,984
  2024: 24,418,603
  2025: 27,039,976


# Overall Quantity Forecasting Implementation

Functions defining the forecasting methods for overall quantity prediction.

## Forecasting Models

We'll implement multiple forecasting approaches:
1. **ARIMA** - Auto-regressive Integrated Moving Average
2. **SARIMA** - Seasonal ARIMA with economic indicators
3. **Exponential Smoothing** - Holt-Winters method
4. **Prophet** - Meta's time series forecasting tool with trend and seasonality
5. **Extra Trees** - Ensemble machine learning method using sliding window features
6. **Ensemble** - Weighted combination of methods

In [1589]:
def forecast_arima(series, steps=12, order=(1,1,1)):
    """
    ARIMA forecasting with automatic order selection if needed
    """
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=steps)
        conf_int = fitted_model.get_forecast(steps=steps).conf_int()
        return forecast, conf_int, fitted_model.aic
    except:
        # Try simpler model if original fails
        try:
            model = ARIMA(series, order=(1,0,1))
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=steps)
            conf_int = fitted_model.get_forecast(steps=steps).conf_int()
            return forecast, conf_int, fitted_model.aic
        except:
            # Last resort - simple naive forecast
            last_value = series.iloc[-1]
            forecast = pd.Series([last_value] * steps)
            conf_int = pd.DataFrame({
                'lower Quantity Invoiced': forecast * 0.9,
                'upper Quantity Invoiced': forecast * 1.1
            })
            return forecast, conf_int, float('inf')

def forecast_sarima(series, steps=12, exog=None, order=(1,1,1), seasonal_order=(1,1,1,12)):
    """
    SARIMA forecasting with external regressors
    """
    try:
        model = SARIMAX(series, exog=exog, order=order, seasonal_order=seasonal_order)
        fitted_model = model.fit(disp=False)
        
        # For forecast, we need future exogenous variables
        # Use last known values as a simple assumption
        if exog is not None:
            future_exog = pd.DataFrame([exog.iloc[-1]] * steps)
            future_exog.index = pd.date_range(start=exog.index[-1] + pd.DateOffset(months=1), periods=steps, freq='MS')
        else:
            future_exog = None
            
        forecast = fitted_model.forecast(steps=steps, exog=future_exog)
        conf_int = fitted_model.get_forecast(steps=steps, exog=future_exog).conf_int()
        return forecast, conf_int, fitted_model.aic
    except:
        # Fallback to simple ARIMA
        return forecast_arima(series, steps, order)

# def forecast_exponential_smoothing(series, steps=12, exog=None, seasonal_periods=12):
#     """
#     Exponential Smoothing (Holt-Winters) forecasting
#     """
#     try:
#         series = deepcopy(series).asfreq('MS')
        
#         if len(series) >= 2 * seasonal_periods:
#             model = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
#         else:
#             model = ExponentialSmoothing(series, trend='add', seasonal=None)
        
#         fitted_model = model.fit()
#         forecast = fitted_model.forecast(steps=steps)
        
#         # Simple confidence intervals based on residuals
#         residuals = fitted_model.resid
#         std_resid = residuals.std()
#         conf_int = pd.DataFrame({
#             'lower Quantity Invoiced': forecast - 1.96 * std_resid,
#             'upper Quantity Invoiced': forecast + 1.96 * std_resid
#         })
        
#         return forecast, conf_int, fitted_model.aic
#     except:
#         # Fallback to ARIMA
        # return forecast_arima(series, steps)
def forecast_exponential_smoothing(series, steps=12, seasonal_periods=12):
    try:
        series = series.asfreq('MS')
        
        if len(series) >= 2 * seasonal_periods:
            model = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
        else:
            model = ExponentialSmoothing(series, trend='add', seasonal=None)

        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps)

        residuals = fitted_model.resid
        std_resid = residuals.std()
        conf_int = pd.DataFrame({
            'lower Quantity Invoiced': forecast - 1.96 * std_resid,
            'upper Quantity Invoiced': forecast + 1.96 * std_resid
        })

        return forecast, conf_int, fitted_model.aic
    except Exception as e:
        print(f"Exponential smoothing failed due to {e}")
        return forecast_arima(series, steps)

def forecast_prophet(series, steps=12, exog=None):
    """
    Prophet forecasting with trend, seasonality, and external regressors
    """
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    prophet_data = pd.DataFrame({
        'ds': series.index,
        'y': series.values
    })
    
    # Initialize Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,  # Monthly data doesn't need weekly seasonality
        daily_seasonality=False,   # Monthly data doesn't need daily seasonality
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,  # Controls flexibility of trend
        seasonality_prior_scale=10.0,  # Controls flexibility of seasonality
        interval_width=0.95
    )
    
    # Add external regressors if provided
    if exog is not None and len(exog.columns) > 0:
        # Add each regressor to the model
        for col in exog.columns:
            if col in ['PP_Spot', 'Resin', 'WTI_Crude_Oil', 'Natural_Gas', 'Energy_Average']:
                # Energy-related indicators tend to have strong impact
                model.add_regressor(col, prior_scale=0.5, mode='multiplicative')
            elif col in ['PMI_Data', 'Factory_Utilization', 'Capacity_Utilization']:
                # Manufacturing indicators
                model.add_regressor(col, prior_scale=0.3, mode='additive')
            else:
                # Other economic indicators
                model.add_regressor(col, prior_scale=0.1, mode='additive')
        
        # Add regressors to prophet_data
        for col in exog.columns:
            prophet_data[col] = exog[col].values
    
    # Fit the model
    model.fit(prophet_data)
    
    # Create future dataframe
    last_date = series.index[-1]
    future_dates = pd.date_range(
        start=last_date + pd.DateOffset(months=1), 
        periods=steps, 
        freq='MS'
    )
    
    future_df = pd.DataFrame({'ds': future_dates})
    
    # Add future regressor values if available
    if exog is not None and len(exog.columns) > 0:
        # Use last known values for future regressors (simple assumption)
        last_regressor_values = exog.iloc[-1]
        for col in exog.columns:
            future_df[col] = last_regressor_values[col]
    
    # Make forecast
    forecast_df = model.predict(future_df)
    
    # Extract forecast values and confidence intervals
    forecast = pd.Series(
        forecast_df['yhat'].values, 
        index=future_dates,
        name='Quantity Invoiced'
    )
    
    conf_int = pd.DataFrame({
        'lower Quantity Invoiced': forecast_df['yhat_lower'].values,
        'upper Quantity Invoiced': forecast_df['yhat_upper'].values
    }, index=future_dates)
    
    # Calculate approximate AIC using cross-validation or residual-based metric
    # Prophet doesn't have built-in AIC, so we'll use MAE on in-sample predictions
    in_sample_forecast = model.predict(prophet_data)
    mae = mean_absolute_error(prophet_data['y'], in_sample_forecast['yhat'])
    pseudo_aic = 2 * len(prophet_data) + 2 * np.log(mae)  # Approximation
    
    return forecast, conf_int, pseudo_aic

def forecast_extra_trees(series, steps=12, exog=None, n_estimators=100, window_size=12):
    """
    Extra Trees forecasting using sliding window approach with external regressors
    """
    # Prepare time series features using sliding window
    def create_time_series_features(ts, window_size, exog_data=None):
        X, y = [], []
        
        for i in range(window_size, len(ts)):
            # Lagged values as features
            lag_features = ts.iloc[i-window_size:i].values
            
            # Add time-based features
            date_idx = ts.index[i]
            time_features = [
                date_idx.month,           # Month of year
                date_idx.quarter,         # Quarter
                date_idx.dayofyear / 365, # Day of year normalized
                np.sin(2 * np.pi * date_idx.month / 12),  # Seasonal sine
                np.cos(2 * np.pi * date_idx.month / 12),  # Seasonal cosine
            ]
            
            # Combine features
            features = list(lag_features) + time_features
            
            # Add exogenous variables if provided
            if exog_data is not None and i < len(exog_data):
                exog_features = exog_data.iloc[i].values.tolist()
                features.extend(exog_features)
            
            X.append(features)
            y.append(ts.iloc[i])
        
        return np.array(X), np.array(y)
    
    # Create training features
    X_train, y_train = create_time_series_features(series, window_size, exog)
    
    if len(X_train) == 0:
        raise ValueError("Not enough data for Extra Trees forecasting")
    
    # Train Extra Trees model
    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        random_state=42,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=True,
        n_jobs=-1
    )
    
    model.fit(X_train, y_train)
    
    # Generate forecasts step by step
    forecast_values = []
    current_series = series.copy()
    
    for step in range(steps):
        # Prepare features for next prediction
        if len(current_series) >= window_size:
            lag_features = current_series.iloc[-window_size:].values
        else:
            # Pad with the last available values if series is too short
            pad_length = window_size - len(current_series)
            padding = [current_series.iloc[-1]] * pad_length
            lag_features = np.array(padding + current_series.tolist())
        
        # Create future date
        next_date = current_series.index[-1] + pd.DateOffset(months=1)
        
        # Time-based features for future date
        time_features = [
            next_date.month,
            next_date.quarter,
            next_date.dayofyear / 365,
            np.sin(2 * np.pi * next_date.month / 12),
            np.cos(2 * np.pi * next_date.month / 12),
        ]
        
        # Combine features
        features = list(lag_features) + time_features
        
        # Add exogenous variables (use last known values)
        if exog is not None:
            exog_features = exog.iloc[-1].values.tolist()
            features.extend(exog_features)
        
        # Make prediction
        next_pred = model.predict([features])[0]
        forecast_values.append(next_pred)
        
        # Update series with new prediction for next iteration
        current_series = pd.concat([
            current_series, 
            pd.Series([next_pred], index=[next_date])
        ])
    
    # Create forecast series with proper index
    future_dates = pd.date_range(
        start=series.index[-1] + pd.DateOffset(months=1),
        periods=steps,
        freq='MS'
    )
    
    forecast = pd.Series(forecast_values, index=future_dates, name='Quantity Invoiced')
    
    # Create confidence intervals based on model's prediction variance
    # For Extra Trees, we can use the std of predictions from individual trees
    predictions_all_trees = np.array([tree.predict([features]) for tree in model.estimators_])
    prediction_std = np.std(predictions_all_trees.flatten()) if len(predictions_all_trees) > 1 else forecast.std()
    
    conf_int = pd.DataFrame({
        'lower Quantity Invoiced': forecast - 1.96 * prediction_std,
        'upper Quantity Invoiced': forecast + 1.96 * prediction_std
    }, index=future_dates)
    
    # Calculate pseudo-AIC based on out-of-bag error
    oob_score = model.score(X_train, y_train) if hasattr(model, 'score') else 0.5
    pseudo_aic = 2 * len(X_train[0]) - 2 * np.log(max(oob_score, 0.001))
    
    return forecast, conf_int, pseudo_aic
        

# def ensemble_forecast(forecasts, aics=None):
#     """
#     Create ensemble forecast from multiple methods (weighted by inverse AIC)
#     """
#     weights = []

#     if aics is None:
#         weights = [1/len(forecasts)] * len(forecasts)
#     else:
#         weights = [1/aic if aic != float('inf') else 0 for aic in aics]
#         total_weight = sum(weights)
#         if total_weight > 0:
#             weights = [w/total_weight for w in weights]
#         else:
#             # Equal weights if all AICs are infinite
#             weights = [1/len(forecasts)] * len(forecasts)

#     # If forecast index is not aligned, reindex to the first forecast's index
#     first_index = forecasts[0].index
#     for i in range(len(forecasts)):
#         if not forecasts[i].index.equals(first_index):
#             forecasts[i] = forecasts[i].reindex(first_index)

    # # Print weights
    # method_names = ['ARIMA', 'SARIMA', 'EXP', 'Prophet', 'ExtraTrees']
    # if len(weights) == len(method_names):
    #     weight_str = ", ".join([f"{name}: {weight:.3f}" for name, weight in zip(method_names, weights)])
    #     print(f"Model weights - {weight_str}")
    # else:
    #     print(f"Model weights: {[f'{w:.3f}' for w in weights]}")
    
    # ensemble = sum(f * w for f, w in zip(forecasts, weights))

    # return ensemble

def ensemble_forecast(forecasts, aics=None, method_names=None, use_median=False):
    """
    Create ensemble forecast from multiple methods
    - If use_median=True, takes the median across methods
    - Otherwise, uses inverse-AIC weighting (default)
    """
    import numpy as np

    if method_names is None:
        method_names = [f"Method_{i+1}" for i in range(len(forecasts))]

    # Align forecast indices
    first_index = forecasts[0].index
    for i in range(len(forecasts)):
        if not forecasts[i].index.equals(first_index):
            forecasts[i] = forecasts[i].reindex(first_index)

    if use_median:
        print(f"Using median ensemble from models: {method_names}")
        ensemble = pd.concat(forecasts, axis=1).median(axis=1)
        return ensemble

    # Default: inverse-AIC weighted ensemble
    if aics is None:
        weights = [1 / len(forecasts)] * len(forecasts)
    else:
        weights = [1 / aic if aic != float('inf') else 0 for aic in aics]
        total_weight = sum(weights)
        weights = [w / total_weight if total_weight else 1 / len(forecasts) for w in weights]

    if len(weights) == len(method_names):
        weight_str = ", ".join([f"{name}: {weight:.3f}" for name, weight in zip(method_names, weights)])
        print(f"Model weights - {weight_str}")
    else:
        print(f"Model weights: {[f'{w:.3f}' for w in weights]}")

    ensemble = sum(f * w for f, w in zip(forecasts, weights))
    return ensemble

    
    print("AIC values:")
    print(f"SARIMA AIC: {overall_sarima_aic}")
    print(f"Prophet pseudo-AIC: {overall_prophet_aic}")
    print(f"Extra Trees pseudo-AIC: {overall_extra_trees_aic}")

    aics = np.array([overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic])
    weights = np.where(aics != float('inf'), 1/aics, 0)
    total_weight = np.sum(weights)
    weights_normalized = weights / total_weight if total_weight else np.ones_like(weights) / len(weights)

    for method, weight in zip(['SARIMA', 'Prophet', 'ExtraTrees'], weights_normalized):
        print(f"{method}: {weight:.4%}")

print("Forecasting functions defined successfully!")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1591, Finished, Available, Finished)

Forecasting functions defined successfully!


## Calculate Accuracy Metrics

In [1590]:
def calculate_accuracy_metrics(actual, predicted, method_name):
    """Calculate comprehensive accuracy metrics"""
    # Remove any NaN values
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    actual_clean = actual[mask]
    predicted_clean = predicted[mask]
    
    if len(actual_clean) == 0:
        return None
    
    # Calculate metrics
    mae = mean_absolute_error(actual_clean, predicted_clean)
    mse = mean_squared_error(actual_clean, predicted_clean)
    rmse = np.sqrt(mse)
    
    # Avoid division by zero for MAPE
    mape = np.mean(np.abs((actual_clean - predicted_clean) / np.where(actual_clean != 0, actual_clean, 1))) * 100
    
    # Additional metrics
    mean_actual = np.mean(actual_clean)
    mean_predicted = np.mean(predicted_clean)
    bias = np.mean(predicted_clean - actual_clean)
    bias_pct = (bias / mean_actual) * 100 if mean_actual != 0 else 0
    
    # R-squared
    ss_res = np.sum((actual_clean - predicted_clean) ** 2)
    ss_tot = np.sum((actual_clean - mean_actual) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
    
    return {
        'Method': method_name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'Bias': bias,
        'Bias_Percent': bias_pct,
        'Mean_Actual': mean_actual,
        'Mean_Predicted': mean_predicted,
        'Data_Points': len(actual_clean)
    }

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1592, Finished, Available, Finished)

In [1591]:
# def forecast_overall(data, exog_vars, forecast_steps, forecast_dates):
#     """
#     Generate overall forecasts using multiple methods and ensemble approach
    
#     Parameters:
#     - data: DataFrame with overall time series data
#     - exog_vars: List of exogenous variables to use in forecasting
#     - forecast_steps: Number of steps to forecast
#     - forecast_dates: Date range for forecasts
    
#     Returns:
#     - DataFrame with all forecast methods and ensemble result
#     """    
#     # Prepare overall data
#     overall_series = data.set_index('Date')['Quantity Invoiced']
#     display(data.head())
#     overall_exog = data.set_index('Date')[exog_vars]
    
#     # Generate forecasts using different methods
#     print("Generating ARIMA forecast...")
#     overall_arima_forecast, overall_arima_conf, overall_arima_aic = forecast_arima(overall_series, forecast_steps)
    
#     print("Generating SARIMA forecast...")
    # overall_sarima_forecast, overall_sarima_conf, overall_sarima_aic = forecast_sarima(overall_series, forecast_steps, overall_exog)
    
    # print("Generating Exponential Smoothing forecast...")
    # overall_exp_forecast, overall_exp_conf, overall_exp_aic = forecast_exponential_smoothing(overall_series, forecast_steps)  
    
    # print("Generating Prophet forecast...")
    # overall_prophet_forecast, overall_prophet_conf, overall_prophet_aic = forecast_prophet(overall_series, forecast_steps, overall_exog)
    
    # print("Generating Extra Trees forecast...")
    # overall_extra_trees_forecast, overall_extra_trees_conf, overall_extra_trees_aic = forecast_extra_trees(overall_series, forecast_steps, overall_exog)

    
    # # Print exponential smoothing forecast
    # print("Exponential Smoothing Forecast:")
    # # print(overall_exp_forecast.head())

    # # Create ensemble forecast
    # # aics = [overall_arima_aic, overall_sarima_aic, overall_exp_aic, overall_prophet_aic, overall_extra_trees_aic]    
    # # overall_ensemble_forecast = ensemble_forecast(
    # #     [overall_arima_forecast, overall_sarima_forecast, overall_exp_forecast, overall_prophet_forecast, overall_extra_trees_forecast], 
    # #     aics
    # # )

    # # Exclude ARIMA
    # # aics = [overall_sarima_aic, overall_exp_aic, overall_prophet_aic, overall_extra_trees_aic]
    # # forecasts = [overall_sarima_forecast, overall_exp_forecast, overall_prophet_forecast, overall_extra_trees_forecast]

    # # overall_ensemble_forecast = ensemble_forecast(
    # #     forecasts, aics, method_names=['SARIMA', 'EXP', 'Prophet', 'ExtraTrees'])

    # # Exclude ARIMA and exp smoothing
    # # aics = [overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic]
    # # forecasts = [overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast]
    # # overall_ensemble_forecast = ensemble_forecast(forecasts, aics, method_names=['SARIMA', 'Prophet', 'ExtraTrees'])

    # # print("Forecast Method Value Lengths:")
    # # print(f"  ARIMA: {len(overall_arima_forecast)}, SARIMA: {len(overall_sarima_forecast)}, "
    # #     f"ExpSmoothing: {len(overall_exp_forecast)}, Prophet: {len(overall_prophet_forecast)}, "
    # #     f"ExtraTrees: {len(overall_extra_trees_forecast)}, Ensemble: {len(overall_ensemble_forecast)}")

    # # Remove ARIMA and ExpSmoothing from ensemble
    # # aics = [overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic]    
    # # overall_ensemble_forecast = ensemble_forecast([overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast], aics)

    # # aics = [overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic]
    # # overall_ensemble_forecast = ensemble_forecast([overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast],aics,method_names=["SARIMA", "Prophet", "ExtraTrees"])
    # aics = [overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic]
    # overall_ensemble_forecast = ensemble_forecast(
    # [overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast],
    # aics,
    # method_names=["SARIMA", "Prophet", "ExtraTrees"])
    # # Store overall forecasts
    # # overall_forecasts = pd.DataFrame({
    # #     'Date': forecast_dates,
    # #     'ARIMA': overall_arima_forecast.values,
    # #     'SARIMA': overall_sarima_forecast.values,
    # #     'ExpSmoothing': overall_exp_forecast.values,
    # #     'Prophet': overall_prophet_forecast.values,
    # #     'ExtraTrees': overall_extra_trees_forecast.values,
    # #     'Ensemble': overall_ensemble_forecast.values,
    # #     'Level': 'Overall',
    # #     'Segment': 'Total'
    # # })
    # overall_forecasts = pd.DataFrame({
    # 'Date': forecast_dates,
    # 'SARIMA': overall_sarima_forecast.values,
#     'Prophet': overall_prophet_forecast.values,
#     'ExtraTrees': overall_extra_trees_forecast.values,
#     'Ensemble': overall_ensemble_forecast.values,
#     'Level': 'Overall',
#     'Segment': 'Total'
# })

#     print(f"Overall forecast range: {overall_ensemble_forecast.min():,.0f} - {overall_ensemble_forecast.max():,.0f}")
    
#     return overall_forecasts

def forecast_overall(data, exog_vars, forecast_steps, forecast_dates):
    """
    Generate overall forecasts using multiple methods and ensemble approach
    """
    # Prepare data
    overall_series = data.set_index('Date')['Quantity Invoiced']
    overall_exog = data.set_index('Date')[exog_vars]

    # Forecasts
    print("Generating SARIMA forecast...")
    overall_sarima_forecast, overall_sarima_conf, overall_sarima_aic = forecast_sarima(overall_series, forecast_steps, overall_exog)

    print("Generating Prophet forecast...")
    overall_prophet_forecast, overall_prophet_conf, overall_prophet_aic = forecast_prophet(overall_series, forecast_steps, overall_exog)

    print("Generating Extra Trees forecast...")
    overall_extra_trees_forecast, overall_extra_trees_conf, overall_extra_trees_aic = forecast_extra_trees(overall_series, forecast_steps, overall_exog)

    # # # Ensemble (only SARIMA, Prophet, Extra Trees)
    # aics = [overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic]
    # overall_ensemble_forecast = ensemble_forecast([overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast], aics, method_names=["SARIMA", "Prophet", "ExtraTrees"])
    
    # New Ensemble Forecast (without ARIMA & ExpSmoothing)
    forecasts = [overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast]
    aics = [overall_sarima_aic, overall_prophet_aic, overall_extra_trees_aic]

    

    print(f"SARIMA AIC: {overall_sarima_aic}")
    print(f"Prophet AIC: {overall_prophet_aic}")
    print(f"Extra Trees AIC: {overall_extra_trees_aic}")

    # overall_ensemble_forecast = ensemble_forecast(
    #     forecasts, 
    #     aics, 
    #     method_names=['SARIMA', 'Prophet', 'ExtraTrees']
    # )

    overall_ensemble_forecast = ensemble_forecast(
    [overall_sarima_forecast, overall_prophet_forecast, overall_extra_trees_forecast],
    method_names=["SARIMA", "Prophet", "ExtraTrees"],
    use_median=True)  # median mode



    print("Overall ensemble forecast generated (ARIMA & ExpSmoothing excluded)")

    # Return all forecasts
    overall_forecasts = pd.DataFrame({
        'Date': forecast_dates,
        'SARIMA': overall_sarima_forecast.values,
        'Prophet': overall_prophet_forecast.values,
        'ExtraTrees': overall_extra_trees_forecast.values,
        'Ensemble': overall_ensemble_forecast.values,
        'Level': 'Overall',
        'Segment': 'Total'
    })

    return overall_forecasts


StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1593, Finished, Available, Finished)

# Running Overall Quantity Forecasting

Execute the overall quantity forecasting using multiple methods and ensemble approach.

In [1592]:
# Set forecasting parameters
FORECAST_STEPS = 12  # 12 months ahead
START_DATE = data['Date'].max()
print(f"Forecasting from: {START_DATE} for {FORECAST_STEPS} steps")
FORECAST_DATES = pd.date_range(start=START_DATE, periods=FORECAST_STEPS, freq='MS')

# Generate overall forecasts
overall_forecasts = forecast_overall(
    data=data,
    exog_vars=exog_vars,
    forecast_steps=FORECAST_STEPS,
    forecast_dates=FORECAST_DATES
)

overall_forecasts

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1594, Finished, Available, Finished)

Forecasting from: 2025-02-01 00:00:00 for 12 steps
Generating SARIMA forecast...


Generating Prophet forecast...
Generating Extra Trees forecast...


SARIMA AIC: 3617.72873992885
Prophet AIC: 272.6110557106993
Extra Trees AIC: 58.15257125139584
Using median ensemble from models: ['SARIMA', 'Prophet', 'ExtraTrees']
Overall ensemble forecast generated (ARIMA & ExpSmoothing excluded)


,Date,SARIMA,Prophet,ExtraTrees,Ensemble,Level,Segment
0,2025-02-01,2.932638e+07,2.664028e+07,2.667783e+07,2.667783e+07,Overall,Total
1,2025-03-01,2.618110e+07,2.279461e+07,2.576553e+07,2.576553e+07,Overall,Total
2,2025-04-01,2.592356e+07,2.309692e+07,2.437550e+07,2.437550e+07,Overall,Total
3,2025-05-01,2.560405e+07,2.632921e+07,2.489365e+07,2.560405e+07,Overall,Total
4,2025-06-01,3.027201e+07,2.619068e+07,2.607862e+07,2.619068e+07,Overall,Total
5,2025-07-01,2.865432e+07,2.456349e+07,2.593749e+07,2.593749e+07,Overall,Total
6,2025-08-01,2.879960e+07,2.716155e+07,2.665838e+07,2.716155e+07,Overall,Total
7,2025-09-01,2.664684e+07,2.353766e+07,2.489142e+07,2.489142e+07,Overall,Total
8,2025-10-01,2.543880e+07,2.288650e+07,2.375679e+07,2.375679e+07,Overall,Total
9,2025-11-01,1.724600e+07,1.607124e+07,1.695239e+07,1.695239e+07,Overall,Total


## Overall Forecast Visualization & Results

Let's visualize the overall forecasts and compare them with historical data.

In [1593]:
# Create comprehensive forecast visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Overall Forecast: Historical vs Predicted', 'Forecast Methods Comparison', 
                    'Forecast Trend Analysis', 'Forecast Method Performance'),
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
    specs=[[{"colspan": 2}, None],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Overall forecast plot (spans both columns in first row)
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Quantity Invoiced'],
              mode='lines+markers', name='Historical Total',
              line=dict(color='darkblue', width=2),
              marker=dict(size=4)),
    row=1, col=1
)

# Add forecast methods
# forecast_colors = {'ARIMA': 'red', 'SARIMA': 'green', 'ExpSmoothing': 'orange', 'Prophet': 'purple', 'ExtraTrees': 'cyan', 'Ensemble': 'black'}
forecast_colors = {'SARIMA': 'green', 'Prophet': 'purple', 'ExtraTrees': 'cyan', 'Ensemble': 'black'}

for method, color in forecast_colors.items():
    if method in overall_forecasts.columns:
        fig.add_trace(
            go.Scatter(x=overall_forecasts['Date'], y=overall_forecasts[method],
                      mode='lines+markers', name=f'{method} Forecast',
                      line=dict(color=color, width=3 if method == 'Ensemble' else 2, 
                               dash='solid' if method == 'Ensemble' else 'dash'),
                      marker=dict(size=6 if method == 'Ensemble' else 4)),
            row=1, col=1
        )

# Method comparison - just the forecast values
# forecast_methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']
# method_colors = ['red', 'green', 'orange', 'purple', 'cyan', 'black']
forecast_methods = ['SARIMA', 'Prophet', 'ExtraTrees', 'Ensemble']
method_colors = ['green', 'purple', 'cyan', 'black']

for method, color in zip(forecast_methods, method_colors):
    if method in overall_forecasts.columns:
        fig.add_trace(
            go.Scatter(x=overall_forecasts['Date'], y=overall_forecasts[method],
                      mode='lines+markers', name=f'{method}',
                      line=dict(color=color, width=2),
                      marker=dict(size=5),
                      showlegend=False),
            row=2, col=1
        )

# Forecast statistics by method
if all(method in overall_forecasts.columns for method in forecast_methods):
    method_stats = []
    for method in forecast_methods:
        mean_val = overall_forecasts[method].mean()
        std_val = overall_forecasts[method].std()
        method_stats.append({'Method': method, 'Mean': mean_val, 'Std': std_val})
    
    stats_df = pd.DataFrame(method_stats)
    
    fig.add_trace(
        go.Bar(x=stats_df['Method'], y=stats_df['Mean'],
               name='Mean Forecast',
               marker_color=['red', 'green', 'orange', 'purple', 'cyan', 'black'],
               showlegend=False),
        row=2, col=2
    )

fig.update_layout(
    height=800,
    title_text="Overall Sales Quantity Forecasting: Historical vs Predicted (12-Month Forecast with Extra Trees)",
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02),
    template="plotly_white"
)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Method", row=2, col=2)

fig.update_yaxes(title_text="Total Quantity", row=1, col=1)
fig.update_yaxes(title_text="Total Quantity", row=2, col=1)
fig.update_yaxes(title_text="Mean Forecast", row=2, col=2)

fig.show()

# Print forecast summary
print("\n=== OVERALL FORECAST SUMMARY ===")
print(f"Forecast Period: {FORECAST_DATES[0].strftime('%Y-%m')} to {FORECAST_DATES[-1].strftime('%Y-%m')}")
print(f"\nOverall Forecast Summary:")
print(f"  Mean Monthly Forecast: {overall_forecasts['Ensemble'].mean():,.0f}")
print(f"  Total 12-Month Forecast: {overall_forecasts['Ensemble'].sum():,.0f}")
print(f"  Min-Max Range: {overall_forecasts['Ensemble'].min():,.0f} - {overall_forecasts['Ensemble'].max():,.0f}")

# Historical comparison
historical_mean = data['Quantity Invoiced'].mean()
historical_total_last_12 = data['Quantity Invoiced'].tail(12).sum()

print(f"\nHistorical Comparison:")
print(f"  Historical Mean Monthly: {historical_mean:,.0f}")
print(f"  Historical Last 12-Month Total: {historical_total_last_12:,.0f}")
print(f"  Forecast vs Historical Mean: {((overall_forecasts['Ensemble'].mean() / historical_mean - 1) * 100):+.1f}%")
print(f"  Forecast vs Last 12-Month Total: {((overall_forecasts['Ensemble'].sum() / historical_total_last_12 - 1) * 100):+.1f}%")

# Method comparison
print(f"\nMethod Comparison (12-Month Totals):")
for method in forecast_methods:
    if method in overall_forecasts.columns:
        total_forecast = overall_forecasts[method].sum()
        vs_ensemble = ((total_forecast / overall_forecasts['Ensemble'].sum() - 1) * 100)
        print(f"  {method}: {total_forecast:,.0f} ({vs_ensemble:+.1f}% vs Ensemble)")

# Growth trajectory
forecast_growth = ((overall_forecasts['Ensemble'].iloc[-1] / overall_forecasts['Ensemble'].iloc[0] - 1) * 100)
print(f"  Forecast Growth (First to Last Month): {forecast_growth:+.1f}%")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1595, Finished, Available, Finished)


=== OVERALL FORECAST SUMMARY ===
Forecast Period: 2025-02 to 2026-01

Overall Forecast Summary:
  Mean Monthly Forecast: 24,937,729
  Total 12-Month Forecast: 299,252,748
  Min-Max Range: 16,952,392 - 27,161,552

Historical Comparison:
  Historical Mean Monthly: 25,717,842
  Historical Last 12-Month Total: 295,807,495
  Forecast vs Historical Mean: -3.0%
  Forecast vs Last 12-Month Total: +1.2%

Method Comparison (12-Month Totals):
  SARIMA: 324,550,712 (+8.5% vs Ensemble)
  Prophet: 289,095,909 (-3.4% vs Ensemble)
  ExtraTrees: 297,927,109 (-0.4% vs Ensemble)
  Ensemble: 299,252,748 (+0.0% vs Ensemble)
  Forecast Growth (First to Last Month): -2.3%


## Export Results

Let's save the overall forecasts to CSV files for further analysis and reporting.

In [1594]:
# Export forecast results
print("=== EXPORTING OVERALL FORECAST RESULTS ===")

# Overall forecast export
# overall_export = overall_forecasts[['Date', 'ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']].copy()
overall_export = overall_forecasts[['Date', 'SARIMA', 'Prophet', 'ExtraTrees', 'Ensemble']].copy()

overall_export.to_csv(data_location + 'modelGeneratedData/overall_sales_forecast.csv', index=False)
print(f"Overall forecasts exported to: overall_sales_forecast.csv")

# Summary report
summary_report = []
summary_report.append({
    'Level': 'Overall',
    'Segment': 'Total',
    'Total_12Month_Forecast': overall_forecasts['Ensemble'].sum(),
    'Average_Monthly_Forecast': overall_forecasts['Ensemble'].mean(),
    'Min_Monthly_Forecast': overall_forecasts['Ensemble'].min(),
    'Max_Monthly_Forecast': overall_forecasts['Ensemble'].max(),
    'Forecast_Growth_Rate': ((overall_forecasts['Ensemble'].iloc[-1] / overall_forecasts['Ensemble'].iloc[0] - 1) * 100),
    'Vs_Historical_Mean': ((overall_forecasts['Ensemble'].mean() / data['Quantity Invoiced'].mean() - 1) * 100)
})

# Add method-specific summaries
# for method in ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees']:
for method in ['SARIMA', 'Prophet', 'ExtraTrees']:
    if method in overall_forecasts.columns:
        summary_report.append({
            'Level': 'Overall',
            'Segment': f'Total_{method}',
            'Total_12Month_Forecast': overall_forecasts[method].sum(),
            'Average_Monthly_Forecast': overall_forecasts[method].mean(),
            'Min_Monthly_Forecast': overall_forecasts[method].min(),
            'Max_Monthly_Forecast': overall_forecasts[method].max(),
            'Forecast_Growth_Rate': ((overall_forecasts[method].iloc[-1] / overall_forecasts[method].iloc[0] - 1) * 100),
            'Vs_Historical_Mean': ((overall_forecasts[method].mean() / data['Quantity Invoiced'].mean() - 1) * 100)
        })

summary_df = pd.DataFrame(summary_report)
summary_df.to_csv(data_location + 'modelGeneratedData/forecast_metrics_summary.csv', index=False)
print(f"Forecast summary report exported to: forecast_metrics_summary.csv")

# Create detailed monthly forecast breakdown
monthly_breakdown = overall_forecasts.copy()
monthly_breakdown['Year'] = monthly_breakdown['Date'].dt.year
monthly_breakdown['Month'] = monthly_breakdown['Date'].dt.month
monthly_breakdown['Month_Name'] = monthly_breakdown['Date'].dt.month_name()
monthly_breakdown['Quarter'] = monthly_breakdown['Date'].dt.quarter

# Calculate quarterly summaries
quarterly_summary = monthly_breakdown.groupby(['Year', 'Quarter']).agg({
    # 'ARIMA': 'sum',
    'SARIMA': 'sum', 
    # 'ExpSmoothing': 'sum',
    'Prophet': 'sum',
    'ExtraTrees': 'sum',
    'Ensemble': 'sum'
}).reset_index()

# quarterly_summary = monthly_breakdown.groupby(['Year', 'Quarter']).agg({
#     'SARIMA': 'sum', 
#     'Prophet': 'sum',
#     'ExtraTrees': 'sum',
#     'Ensemble': 'sum'
# }).reset_index()

quarterly_summary['Quarter_Label'] = quarterly_summary['Year'].astype(str) + '-Q' + quarterly_summary['Quarter'].astype(str)

quarterly_summary.to_csv(data_location + 'modelGeneratedData/quarterly_forecast_summary.csv', index=False)
print(f"Quarterly forecast summary exported to: quarterly_forecast_summary.csv")

print(f"\n=== EXPORT COMPLETE ===")
print(f"Total files exported: 3")
print(f"Export timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Display final summary
print(f"\n=== FINAL FORECAST SUMMARY ===")
display(summary_df)

print(f"\n=== QUARTERLY FORECAST BREAKDOWN ===")
# display(quarterly_summary[['Quarter_Label', 'ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']])
display(quarterly_summary[['Quarter_Label', 'SARIMA', 'Prophet', 'ExtraTrees', 'Ensemble']])


StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1596, Finished, Available, Finished)

=== EXPORTING OVERALL FORECAST RESULTS ===
Overall forecasts exported to: overall_sales_forecast.csv
Forecast summary report exported to: forecast_metrics_summary.csv
Quarterly forecast summary exported to: quarterly_forecast_summary.csv

=== EXPORT COMPLETE ===
Total files exported: 3
Export timestamp: 2025-08-02 04:02:39

=== FINAL FORECAST SUMMARY ===


SynapseWidget(Synapse.DataFrame, 60a6468e-e329-4ae1-8d06-35bc9c0400da)


=== QUARTERLY FORECAST BREAKDOWN ===


SynapseWidget(Synapse.DataFrame, 9abf219d-3dc2-49a5-9c3a-5b7ec0ae76c2)

## 2025 Actuals Comparison

In [1595]:
# Compute the total Quantity Invoiced for each month of 2025
# 2025 Actuals Comparison so far
print("\n=== 2025 ACTUALS COMPARISON ===")

# Loading Quantity data in pandas DataFrames
# 2015 to 2025 Qty.csv
actual_quantity_df = pd.read_csv(data_location + "userProvidedData/2015-2025 Qty.csv", parse_dates=['Fiscal Hierarchy - Full Date'])
actual_quantity_df = actual_quantity_df[actual_quantity_df['Fiscal Hierarchy - Full Date'].dt.year == 2025]

# Ensure Quantity Invoiced is numeric
actual_quantity_df['Quantity Invoiced'] = actual_quantity_df['Quantity Invoiced'].astype(str).str.replace(',', '')
actual_quantity_df['Quantity Invoiced'] = pd.to_numeric(actual_quantity_df['Quantity Invoiced'], errors='coerce')

# # Filtering Later Quantity Data to match
# print(f"Before filtering quantity data shape: {actual_quantity_df.shape}")

# # Filter to InterCompany
# actual_quantity_df = actual_quantity_df[actual_quantity_df['CustomerSegment'] == 'InterCompany']
# print(f"After filtering to InterCompany segment, quantity data shape: {actual_quantity_df.shape}")

# Group by month and sum the Quantity Invoiced
actual_quantity_df = actual_quantity_df.groupby(actual_quantity_df['Fiscal Hierarchy - Full Date'].dt.to_period('M'))['Quantity Invoiced'].sum().reset_index()
actual_quantity_df['Fiscal Hierarchy - Full Date'] = actual_quantity_df['Fiscal Hierarchy - Full Date'].dt.to_timestamp()
# Rename columns for clarity
actual_quantity_df.rename(columns={'Fiscal Hierarchy - Full Date': 'Date', 'Quantity Invoiced': 'Actual_Quantity'}, inplace=True)

# Display the monthly actuals for 2025
display(actual_quantity_df.head())


StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1597, Finished, Available, Finished)


=== 2025 ACTUALS COMPARISON ===


SynapseWidget(Synapse.DataFrame, de46507a-3b6c-4e30-af64-a7522f79eb6e)

In [1596]:
# Create a CSV file with 2025 actuals and forecast comparison by combining with overall_export
comparison_df = overall_forecasts.merge(actual_quantity_df, on='Date', how='left')
# Drop the 'Level' and 'Segment' columns from overall_export
comparison_df = comparison_df.drop(columns=['Level', 'Segment'])
# Only use rows where Actual_Quantity is not NaN
comparison_df = comparison_df[~comparison_df['Actual_Quantity'].isna()]
# Export as CSV
comparison_df.to_csv(data_location + 'modelGeneratedData/2025_actuals_forecast_comparison.csv', index=False)
print(f"2025 actuals vs forecast comparison exported to: 2025_actuals_forecast_comparison.csv")

# forecast_methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees']
forecast_methods = ['SARIMA', 'Prophet', 'ExtraTrees']


# Display comparison df
display(comparison_df)

# Calculate accuracy metrics for each method
accuracy_results = []
for method in forecast_methods:
    if method in comparison_df.columns:
        metrics = calculate_accuracy_metrics(
            comparison_df['Actual_Quantity'].values,
            comparison_df[method].values,
            method
        )
        if metrics:
            accuracy_results.append(metrics)

# Convert results to DataFrame and display
accuracy_df = pd.DataFrame(accuracy_results)
display(accuracy_df)

# Export accuracy metrics to CSV
accuracy_df.to_csv(data_location + 'modelGeneratedData/2025_forecast_accuracy_metrics.csv', index=False)

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1598, Finished, Available, Finished)

2025 actuals vs forecast comparison exported to: 2025_actuals_forecast_comparison.csv


SynapseWidget(Synapse.DataFrame, 581ef537-2cf2-409e-92d1-b5310320c19f)

SynapseWidget(Synapse.DataFrame, dc82f9e1-8111-48da-9b6e-03f6c50b45c1)

In [1597]:
# Plot forcasts methods vs actuals
fig = go.Figure()
# Plot Actual Quantity
fig.add_trace(
    go.Scatter(x=comparison_df['Date'], y=comparison_df['Actual_Quantity'],
              mode='lines+markers', name='Actual Quantity',
              line=dict(color='black', width=2),
              marker=dict(size=4))
)

# Plot each forecast method as a dotted line
# forecast_methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees']
forecast_methods = ['SARIMA', 'Prophet', 'ExtraTrees']
for method in forecast_methods:
    if method in comparison_df.columns:
        fig.add_trace(
            go.Scatter(
                x=comparison_df['Date'],
                y=comparison_df[method],
                mode='lines+markers',
                name=f'{method} Forecast',
                line=dict(width=2, dash='dot'),
                marker=dict(size=4)
            )
        )

fig.update_layout(
    title="2025 Actuals vs Forecast Methods (Including Extra Trees)",
    xaxis_title="Date",
    yaxis_title="Quantity",
    template="plotly_white"
)

# Show the plot
fig.show()

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1599, Finished, Available, Finished)

# Backtesting Analysis

Let's evaluate the performance of our forecasting models by backtesting on previous data. For 2024, we'll train models on data up to 2023 and test predictions against actual 2024 values.

In [1598]:
# Backtesting Setup: Split data into train (2015-2023) and test (2024)
print("=== BACKTESTING SETUP ===")

# Define split date
BACKTEST_SPLIT_DATE = pd.to_datetime('2024-01-01') #aaa
# Forecast horizon
FORECAST_HORIZON = 3  # 12 or 3
print(f"Training data: Before {BACKTEST_SPLIT_DATE}")
print(f"Test data: {BACKTEST_SPLIT_DATE} onwards")

# Check if we have test data
data_test = data[(data['Date'] >= BACKTEST_SPLIT_DATE) & (data['Date'] < BACKTEST_SPLIT_DATE + pd.DateOffset(months=FORECAST_HORIZON))]
data_train = data[data['Date'] < BACKTEST_SPLIT_DATE]

# Display training and test data columns
print(f"Training data columns: {data_train.columns.tolist()}")
print(f"Test data columns: {data_test.columns.tolist()}")

print(f"Total data points: {len(data)}")
print(f"Training data points: {len(data_train)}")
print(f"Test data points: {len(data_test)}")
print(f"Test Data range: {data_test['Date'].min()} to {data_test['Date'].max()}")

if len(data_test) == 0:
    print("⚠️ No data available for backtesting!")
else:
    print(f"✅ Found {len(data_test)} data points for backtesting")
    print(f"Test months available: {sorted(data_test['Date'].dt.strftime('%Y-%m').unique())}")

# Create training overall aggregation
print("\n=== CREATING TRAINING AGGREGATION ===")

# Overall training data
print(f"Training data shape: {data_train.shape}")
print(f"Training period: {data_train['Date'].min()} to {data_train['Date'].max()}")

# Create actual aggregations for comparison
print("\n=== CREATING ACTUAL AGGREGATIONS ===")

if len(data_test) > 0:
    # Overall actuals
    overall_actual = data_test.groupby('Date').agg({
        'Quantity Invoiced': 'sum',
    }).reset_index()
    
    print(f"Actual data shape: {overall_actual.shape}")
    print(f"Actual period: {overall_actual['Date'].min()} to {overall_actual['Date'].max()}")
    
    print(f"\nOverall monthly totals:")
    for _, row in overall_actual.iterrows():
        print(f"  {row['Date'].strftime('%Y-%m')}: {row['Quantity Invoiced']:,.0f}")
        
    # Basic statistics
    print(f"\nBacktesting Statistics:")
    print(f"  Training samples: {len(data_train)}")
    print(f"  Test samples: {len(overall_actual)}")
    print(f"  Training mean: {data_train['Quantity Invoiced'].mean():,.0f}")
    print(f"  Test mean: {overall_actual['Quantity Invoiced'].mean():,.0f}")
    print(f"  Training std: {data_train['Quantity Invoiced'].std():,.0f}")
    print(f"  Test std: {overall_actual['Quantity Invoiced'].std():,.0f}")
else:
    print("No data available for comparison")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1600, Finished, Available, Finished)

=== BACKTESTING SETUP ===
Training data: Before 2024-01-01 00:00:00
Test data: 2024-01-01 00:00:00 onwards
Training data columns: ['Date', 'Quantity Invoiced', 'Quantity Invoiced Mean', 'Quantity Invoiced Count', 'Sales - USD', 'Sales - USD Mean', 'Sales - USD Count', 'Unique Customers', 'Unique Products', 'Unique Categories', 'Unique SubCategories', 'Unique EndMarkets L1', 'Unique EndMarkets L2', 'future_orders_count', 'future_orders_qty_total', 'future_orders_qty_next_1m', 'future_orders_qty_next_3m', 'future_orders_qty_next_6m', 'future_orders_qty_next_12m', 'future_orders_sales_next_1m', 'future_orders_sales_next_3m', 'future_orders_sales_next_6m', 'future_orders_sales_next_12m', 'future_orders_avg_lead_time', 'future_orders_min_lead_time', 'future_orders_max_lead_time', 'future_orders_due_next_month', 'future_orders_due_next_quarter', 'future_orders_unique_customers', 'future_orders_unique_products', 'future_to_current_ratio', 'qty_rolling_avg_3m', 'future_to_rolling_3m_ratio', 'q

In [1599]:
# Generate Backtesting Forecasts (Train on 2015-2023, Predict 2024)
if len(data_test) > 0:
    print("=== OVERALL FORECASTING BACKTESTING ===")
    
    # Determine how many months of data we have
    backtest_months = len(overall_actual)
    backtest_dates = pd.date_range(start=BACKTEST_SPLIT_DATE, periods=backtest_months, freq='MS')
    
    print(f"Generating overall forecasts for {backtest_months} months")
    
    # Generate Overall Backtest Forecast using the reusable function
    print("\nGenerating Overall Backtest Forecast...")
    overall_backtest_forecasts = forecast_overall(
        data=data_train,
        exog_vars=exog_vars,
        forecast_steps=backtest_months,
        forecast_dates=backtest_dates
    )
    
    # Store backtest forecasts for overall level
    backtest_forecasts = overall_backtest_forecasts.copy()
    
    # Add actual values
    backtest_forecasts = backtest_forecasts.merge(
        overall_actual[['Date', 'Quantity Invoiced']], 
        on='Date', 
        how='left'
    )
    backtest_forecasts.rename(columns={'Quantity Invoiced': 'Actual'}, inplace=True)
    
    print(f"Backtest forecast range:")
    # print(f"  ARIMA: {backtest_forecasts['ARIMA'].min():,.0f} - {backtest_forecasts['ARIMA'].max():,.0f}")
    print(f"  SARIMA: {backtest_forecasts['SARIMA'].min():,.0f} - {backtest_forecasts['SARIMA'].max():,.0f}")
    # print(f"  ExpSmoothing: {backtest_forecasts['ExpSmoothing'].min():,.0f} - {backtest_forecasts['ExpSmoothing'].max():,.0f}")
    print(f"  Prophet: {backtest_forecasts['Prophet'].min():,.0f} - {backtest_forecasts['Prophet'].max():,.0f}")
    print(f"  ExtraTrees: {backtest_forecasts['ExtraTrees'].min():,.0f} - {backtest_forecasts['ExtraTrees'].max():,.0f}")
    print(f"  Ensemble: {backtest_forecasts['Ensemble'].min():,.0f} - {backtest_forecasts['Ensemble'].max():,.0f}")
    print(f"Actual range: {overall_actual['Quantity Invoiced'].min():,.0f} - {overall_actual['Quantity Invoiced'].max():,.0f}")
    
    print("\nBacktest vs Actual comparison:")
    display(backtest_forecasts)
    
    # # Calculate forecast errors for quick assessment
    # if 'Actual' in backtest_forecasts.columns:
    #     # for method in ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']:
    #     for method in ['SARIMA', 'Prophet', 'ExtraTrees', 'Ensemble']:
    #         if method in backtest_forecasts.columns:
    #             mae = np.mean(np.abs(backtest_forecasts['Actual'] - backtest_forecasts[method]))
    #             mape = np.mean(np.abs((backtest_forecasts['Actual'] - backtest_forecasts[method]) / backtest_forecasts['Actual'])) * 100
    #             print(f"  {method} - MAE: {mae:,.0f}, MAPE: {mape:.2f}%")
    # Calculate forecast errors for quick assessment (excluding ARIMA and ExpSmoothing)
    if 'Actual' in backtest_forecasts.columns:
        for method in ['SARIMA', 'Prophet', 'ExtraTrees', 'Ensemble']:
            if method in backtest_forecasts.columns:
                mae = np.mean(np.abs(backtest_forecasts['Actual'] - backtest_forecasts[method]))
                mape = np.mean(np.abs((backtest_forecasts['Actual'] - backtest_forecasts[method]) / backtest_forecasts['Actual'])) * 100
                print(f"  {method} - MAE: {mae:,.0f}, MAPE: {mape:.2f}%")
    
else:
    print("⚠️ Skipping backtesting - no 2024 data available")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1601, Finished, Available, Finished)

=== OVERALL FORECASTING BACKTESTING ===
Generating overall forecasts for 3 months

Generating Overall Backtest Forecast...
Generating SARIMA forecast...


Generating Prophet forecast...
Generating Extra Trees forecast...


SARIMA AIC: 3155.105990724925
Prophet AIC: 244.4582373901987
Extra Trees AIC: 58.13572417854013
Using median ensemble from models: ['SARIMA', 'Prophet', 'ExtraTrees']
Overall ensemble forecast generated (ARIMA & ExpSmoothing excluded)
Backtest forecast range:
  SARIMA: 25,306,638 - 27,361,623
  Prophet: 17,793,889 - 22,090,503
  ExtraTrees: 21,984,113 - 22,385,882
  Ensemble: 21,999,418 - 22,385,882
Actual range: 25,626,124 - 26,483,836

Backtest vs Actual comparison:


SynapseWidget(Synapse.DataFrame, 50afd30c-b3a8-40b4-95d5-ac6cd91db1f8)

  SARIMA - MAE: 428,081, MAPE: 1.63%
  Prophet - MAE: 6,196,617, MAPE: 23.98%
  ExtraTrees - MAE: 3,803,373, MAPE: 14.64%
  Ensemble - MAE: 3,767,910, MAPE: 14.51%


In [1600]:
# Calculate Backtesting Accuracy Metrics
if len(data_test) > 0:
    print("\n=== BACKTESTING ACCURACY METRICS ===")
    
    # Calculate metrics for each method
    accuracy_results = []
    
    # Check which methods are available in the forecast data
    # methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']
    # method_names = ['ARIMA', 'SARIMA', 'Exponential Smoothing', 'Prophet', 'Extra Trees', 'Ensemble']
    methods = ['SARIMA', 'Prophet', 'ExtraTrees', 'Ensemble']
    method_names = ['SARIMA', 'Prophet', 'Extra Trees', 'Ensemble']

    for method, name in zip(methods, method_names):
        if method in backtest_forecasts.columns:
            metrics = calculate_accuracy_metrics(
                backtest_forecasts['Actual'].values,
                backtest_forecasts[method].values,
                name
            )
            if metrics:
                accuracy_results.append(metrics)
    
    # Convert to DataFrame
    accuracy_df = pd.DataFrame(accuracy_results)
    
    print("Overall Backtesting Accuracy Results:")
    print("="*80)
    for _, row in accuracy_df.iterrows():
        print(f"\n{row['Method']} Performance:")
        print(f"  MAE (Mean Absolute Error): {row['MAE']:,.0f}")
        print(f"  RMSE (Root Mean Square Error): {row['RMSE']:,.0f}")
        print(f"  MAPE (Mean Absolute Percentage Error): {row['MAPE']:.2f}%")
        print(f"  R² (Coefficient of Determination): {row['R2']:.4f}")
        print(f"  Bias: {row['Bias']:,.0f} ({row['Bias_Percent']:.2f}%)")
        print(f"  Mean Actual: {row['Mean_Actual']:,.0f}")
        print(f"  Mean Predicted: {row['Mean_Predicted']:,.0f}")
    
    # Find best performing method
    best_method = accuracy_df.loc[accuracy_df['MAPE'].idxmin(), 'Method']
    best_mape = accuracy_df.loc[accuracy_df['MAPE'].idxmin(), 'MAPE']
    
    print(f"\n🏆 Best Performing Method: {best_method} (MAPE: {best_mape:.2f}%)")
    
    # Export accuracy results
    accuracy_df.to_csv(data_location + 'modelGeneratedData/backtesting_accuracy_metrics.csv', index=False)
    print(f"\n📊 Accuracy metrics exported to: backtesting_accuracy_metrics.csv")
    
    display(accuracy_df)
else:
    print("⚠️ Skipping accuracy calculation - no 2024 data available")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1602, Finished, Available, Finished)


=== BACKTESTING ACCURACY METRICS ===
Overall Backtesting Accuracy Results:

SARIMA Performance:
  MAE (Mean Absolute Error): 428,081
  RMSE (Root Mean Square Error): 541,647
  MAPE (Mean Absolute Percentage Error): 1.63%
  R² (Coefficient of Determination): -0.8852
  Bias: 215,090 (0.83%)
  Mean Actual: 25,926,511
  Mean Predicted: 26,141,601

Prophet Performance:
  MAE (Mean Absolute Error): 6,196,617
  RMSE (Root Mean Square Error): 6,354,772
  MAPE (Mean Absolute Percentage Error): 23.98%
  R² (Coefficient of Determination): -258.4979
  Bias: -6,196,617 (-23.90%)
  Mean Actual: 25,926,511
  Mean Predicted: 19,729,894

Extra Trees Performance:
  MAE (Mean Absolute Error): 3,803,373
  RMSE (Root Mean Square Error): 3,837,670
  MAPE (Mean Absolute Percentage Error): 14.64%
  R² (Coefficient of Determination): -93.6388
  Bias: -3,803,373 (-14.67%)
  Mean Actual: 25,926,511
  Mean Predicted: 22,123,138

Ensemble Performance:
  MAE (Mean Absolute Error): 3,767,910
  RMSE (Root Mean Squar

SynapseWidget(Synapse.DataFrame, 8da634dd-e780-4d4d-b0a6-6a71b3736c1f)

### Backtesting Visualization

In [1601]:
# Backtesting Visualization
if len(data_test) > 0:
    print("\n=== BACKTESTING VISUALIZATION ===")
    
    # Create comprehensive backtesting visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Forecast vs Actual (Overall)', 
            'Forecast Accuracy by Method (MAPE)',
            'Residuals Analysis (Ensemble)',
            'Method Performance Comparison'
        ),
        specs=[[{"secondary_y": False}, {"type": "bar"}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Plot 1: Forecast vs Actual
    fig.add_trace(
        go.Scatter(
            x=backtest_forecasts['Date'], 
            y=backtest_forecasts['Actual'],
            mode='lines+markers', 
            name='Actual',
            line=dict(color='darkblue', width=3),
            marker=dict(size=6)
        ), row=1, col=1
    )
    
    # Add forecast methods
    colors_bt = ['red', 'green', 'orange', 'purple', 'cyan', 'black']
    methods_bt = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']
    method_labels = ['ARIMA', 'SARIMA', 'Exp Smoothing', 'Prophet', 'Extra Trees', 'Ensemble']
    
    for method, label, color in zip(methods_bt, method_labels, colors_bt):
        if method in backtest_forecasts.columns:
            fig.add_trace(
                go.Scatter(
                    x=backtest_forecasts['Date'], 
                    y=backtest_forecasts[method],
                    mode='lines+markers', 
                    name=f'{label} Forecast',
                    line=dict(color=color, width=3 if method == 'Ensemble' else 2, 
                             dash='solid' if method == 'Ensemble' else 'dash'),
                    marker=dict(size=6 if method == 'Ensemble' else 4)
                ), row=1, col=1
            )
    
    # Plot 2: MAPE Comparison
    if len(accuracy_results) > 0:
        fig.add_trace(
            go.Bar(
                x=[r['Method'] for r in accuracy_results],
                y=[r['MAPE'] for r in accuracy_results],
                name='MAPE %',
                marker_color=['red' if r['Method'] == best_method else 'lightblue' for r in accuracy_results],
                showlegend=False
            ), row=1, col=2
        )
    
    # Plot 3: Residuals (Actual - Ensemble Forecast)
    if 'Ensemble' in backtest_forecasts.columns:
        residuals = backtest_forecasts['Actual'] - backtest_forecasts['Ensemble']
        fig.add_trace(
            go.Scatter(
                x=backtest_forecasts['Date'], 
                y=residuals,
                mode='lines+markers', 
                name='Residuals (Actual - Ensemble)',
                line=dict(color='red', width=2),
                marker=dict(size=5),
                showlegend=False
            ), row=2, col=1
        )
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
    
    # Plot 4: Method Performance Metrics
    if len(accuracy_results) > 0:
        methods = [r['Method'] for r in accuracy_results]
        mae_values = [r['MAE'] for r in accuracy_results]
        
        # Create a normalized view of MAE for comparison
        fig.add_trace(
            go.Bar(
                x=methods,
                y=mae_values,
                name='MAE',
                marker_color=['gold' if m == best_method else 'lightcoral' for m in methods],
                showlegend=False
            ), row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Overall Forecasting Backtesting Analysis: Performance & Accuracy Assessment",
        showlegend=True,
        template="plotly_white"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Method", row=1, col=2)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_xaxes(title_text="Method", row=2, col=2)
    
    fig.update_yaxes(title_text="Quantity", row=1, col=1)
    fig.update_yaxes(title_text="MAPE (%)", row=1, col=2)
    fig.update_yaxes(title_text="Residuals", row=2, col=1)
    fig.update_yaxes(title_text="MAE", row=2, col=2)
    
    fig.show()
    
    # Additional Analysis: Directional Accuracy
    if 'Ensemble' in backtest_forecasts.columns and len(backtest_forecasts) > 1:
        print("\n=== DIRECTIONAL ACCURACY ANALYSIS ===")
        
        # Calculate month-over-month changes
        actual_changes = backtest_forecasts['Actual'].diff().dropna()
        forecast_changes = backtest_forecasts['Ensemble'].diff().dropna()
        
        # Calculate directional accuracy (same sign of change)
        directional_accuracy = np.mean((actual_changes * forecast_changes) > 0) * 100
        
        print(f"Directional Accuracy: {directional_accuracy:.1f}%")
        print("(Percentage of time forecast correctly predicted direction of change)")
        
        # Additional directional analysis
        correct_direction = (actual_changes * forecast_changes) > 0
        print(f"\nDirectional Analysis:")
        print(f"  Months with correct direction: {correct_direction.sum()}/{len(correct_direction)}")
        print(f"  Months with incorrect direction: {(~correct_direction).sum()}/{len(correct_direction)}")
        
        # Show specific months with wrong direction
        if (~correct_direction).any():
            wrong_dates = backtest_forecasts['Date'].iloc[1:][~correct_direction]
            print(f"  Months with wrong direction: {', '.join(wrong_dates.dt.strftime('%Y-%m'))}")
        
        # Normality test on residuals
        if len(residuals) > 3:
            from scipy import stats
            try:
                shapiro_stat, shapiro_p = stats.shapiro(residuals)
                print(f"\nResiduals Normality Test (Shapiro-Wilk):")
                print(f"  Test Statistic: {shapiro_stat:.4f}")
                print(f"  P-value: {shapiro_p:.4f}")
                normality_result = "Normal" if shapiro_p > 0.05 else "Not Normal"
                print(f"  Result: {normality_result} at 5% significance level")
            except:
                print(f"\nResiduals Normality Test: Could not be performed")
        
        # Mean Absolute Deviation
        mad = np.mean(np.abs(residuals))
        print(f"\nMean Absolute Deviation: {mad:,.0f}")
        
        # Forecast bias analysis
        bias = np.mean(residuals)
        print(f"Forecast Bias: {bias:,.0f}")
        if bias > 0:
            print("  Interpretation: Forecast tends to underpredict")
        elif bias < 0:
            print("  Interpretation: Forecast tends to overpredict")
        else:
            print("  Interpretation: Forecast is unbiased")
        
        # Percentage bias
        bias_pct = (bias / np.mean(backtest_forecasts['Actual'])) * 100
        print(f"Percentage Bias: {bias_pct:.2f}%")
    
    print("\n=== BACKTESTING VISUALIZATION COMPLETE ===")
    
else:
    print("⚠️ Skipping backtesting visualization - data unavailable")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1603, Finished, Available, Finished)


=== BACKTESTING VISUALIZATION ===



=== DIRECTIONAL ACCURACY ANALYSIS ===
Directional Accuracy: 100.0%
(Percentage of time forecast correctly predicted direction of change)

Directional Analysis:
  Months with correct direction: 2/2
  Months with incorrect direction: 0/2

Mean Absolute Deviation: 3,767,910
Forecast Bias: 3,767,910
  Interpretation: Forecast tends to underpredict
Percentage Bias: 14.53%

=== BACKTESTING VISUALIZATION COMPLETE ===


### Backtesting Summary Report

In [1602]:
# Generate Comprehensive Backtesting Summary Report
if len(data_test) > 0:
    print("\n=== GENERATING BACKTESTING SUMMARY REPORT ===")
    
    # Create comprehensive summary report
    summary_report_data = []
    
    # Overall performance summary
    summary_report_data.append({
        'Section': 'Overall Performance',
        'Metric': 'Best Method',
        'Value': best_method,
        'Details': f'MAPE: {best_mape:.2f}%'
    })
    
    # Add detailed metrics for best method
    best_method_metrics = accuracy_df[accuracy_df['Method'] == best_method].iloc[0]
    
    key_metrics = [
        ('MAE', 'Mean Absolute Error', f"{best_method_metrics['MAE']:,.0f}"),
        ('RMSE', 'Root Mean Square Error', f"{best_method_metrics['RMSE']:,.0f}"),
        ('MAPE', 'Mean Absolute Percentage Error', f"{best_method_metrics['MAPE']:.2f}%"),
        ('R2', 'R-Squared', f"{best_method_metrics['R2']:.4f}"),
        ('Bias', 'Forecast Bias', f"{best_method_metrics['Bias']:,.0f}"),
        ('Bias_Percent', 'Bias Percentage', f"{best_method_metrics['Bias_Percent']:.2f}%")
    ]
    
    for metric_code, metric_name, value in key_metrics:
        summary_report_data.append({
            'Section': 'Best Method Metrics',
            'Metric': metric_name,
            'Value': value,
            'Details': f'{best_method} performance'
        })
    
    # Data coverage
    summary_report_data.append({
        'Section': 'Data Coverage',
        'Metric': 'Training Period',
        'Value': f"{data_train['Date'].min().strftime('%Y-%m')} to {data_train['Date'].max().strftime('%Y-%m')}",
        'Details': f"{len(data_train)} months"
    })
    
    summary_report_data.append({
        'Section': 'Data Coverage',
        'Metric': 'Test Period',
        'Value': f"{data_test['Date'].min().strftime('%Y-%m')} to {data_test['Date'].max().strftime('%Y-%m')}",
        'Details': f"{len(overall_actual)} months"
    })
    
    # Model comparison insights
    if len(accuracy_results) > 1:
        worst_method = accuracy_df.loc[accuracy_df['MAPE'].idxmax(), 'Method']
        worst_mape = accuracy_df.loc[accuracy_df['MAPE'].idxmax(), 'MAPE']
        improvement = worst_mape - best_mape
        
        summary_report_data.append({
            'Section': 'Model Insights',
            'Metric': 'Method Improvement',
            'Value': f"{improvement:.2f}% MAPE reduction",
            'Details': f"{best_method} vs {worst_method}"
        })
        
        # Add all method performances
        for result in accuracy_results:
            summary_report_data.append({
                'Section': 'Method Performance',
                'Metric': f"{result['Method']} MAPE",
                'Value': f"{result['MAPE']:.2f}%",
                'Details': f"MAE: {result['MAE']:,.0f}, R²: {result['R2']:.4f}"
            })
    
    # Statistical significance
    if 'Ensemble' in backtest_forecasts.columns:
        residuals = backtest_forecasts['Actual'] - backtest_forecasts['Ensemble']
        
        # Shapiro-Wilk test for normality of residuals
        try:
            from scipy import stats
            shapiro_stat, shapiro_p = stats.shapiro(residuals)
            normality_result = "Normal" if shapiro_p > 0.05 else "Non-normal"
            
            summary_report_data.append({
                'Section': 'Statistical Tests',
                'Metric': 'Residuals Normality',
                'Value': normality_result,
                'Details': f"Shapiro-Wilk p-value: {shapiro_p:.4f}"
            })
        except:
            summary_report_data.append({
                'Section': 'Statistical Tests',
                'Metric': 'Residuals Normality',
                'Value': "Could not test",
                'Details': "Test not available"
            })
        
        # Mean absolute deviation
        mad = np.mean(np.abs(residuals - np.mean(residuals)))
        summary_report_data.append({
            'Section': 'Statistical Tests',
            'Metric': 'Mean Absolute Deviation',
            'Value': f"{mad:,.0f}",
            'Details': "Residuals spread measure"
        })
        
        # Directional accuracy
        if len(backtest_forecasts) > 1:
            actual_changes = backtest_forecasts['Actual'].diff().dropna()
            forecast_changes = backtest_forecasts['Ensemble'].diff().dropna()
            directional_accuracy = np.mean((actual_changes * forecast_changes) > 0) * 100
            
            summary_report_data.append({
                'Section': 'Directional Analysis',
                'Metric': 'Directional Accuracy',
                'Value': f"{directional_accuracy:.1f}%",
                'Details': "Correct direction prediction rate"
            })
    
    # Forecast characteristics
    summary_report_data.append({
        'Section': 'Forecast Characteristics',
        'Metric': 'Training Data Mean',
        'Value': f"{data_train['Quantity Invoiced'].mean():,.0f}",
        'Details': "Average monthly quantity in training"
    })
    
    summary_report_data.append({
        'Section': 'Forecast Characteristics',
        'Metric': 'Test Data Mean',
        'Value': f"{overall_actual['Quantity Invoiced'].mean():,.0f}",
        'Details': "Average monthly quantity in test period"
    })
    
    if 'Ensemble' in backtest_forecasts.columns:
        summary_report_data.append({
            'Section': 'Forecast Characteristics',
            'Metric': 'Ensemble Forecast Mean',
            'Value': f"{backtest_forecasts['Ensemble'].mean():,.0f}",
            'Details': "Average monthly forecast"
        })
    
    # Create summary DataFrame
    summary_df_report = pd.DataFrame(summary_report_data)
    
    # Export summary report
    summary_df_report.to_csv(data_location + 'modelGeneratedData/backtesting_summary_report.csv', index=False)

    # Create a csv with the actuals and overall forecast of each method for each month
    export_df = backtest_forecasts.copy()
    
    # Clean up columns for export
    export_columns = ['Date', 'Actual']
    for method in ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'ExtraTrees', 'Ensemble']:
        if method in export_df.columns:
            export_columns.append(method)
    
    export_df = export_df[export_columns]
    export_df.to_csv(data_location + 'modelGeneratedData/backtesting_actuals_vs_forecast_by_method.csv', index=False)
    print('Exported actuals and forecasts by method to: backtesting_actuals_vs_forecast_by_method.csv')

    # Display formatted summary
    print(f"\n{'='*80}")
    print(f"                 OVERALL FORECASTING BACKTESTING SUMMARY")
    print(f"{'='*80}")
    
    current_section = ""
    for _, row in summary_df_report.iterrows():
        if row['Section'] != current_section:
            current_section = row['Section']
            print(f"\n🔹 {current_section.upper()}")
            print(f"{'-'*50}")
        
        print(f"  {row['Metric']:<30}: {row['Value']:<20} ({row['Details']})")
    
    print(f"\n{'='*80}")
    print(f"📊 Complete backtesting summary exported to: backtesting_summary_report.csv")
    print(f"🎯 Overall forecasting backtesting analysis completed successfully!")
    
    # Display final summary table
    print(f"\n📋 FINAL BACKTESTING RESULTS:")
    summary_table = accuracy_df[['Method', 'MAPE', 'MAE', 'R2', 'Bias_Percent']].round(2)
    print(summary_table.to_string(index=False))
    
    # Key insights
    print(f"\n🔍 KEY INSIGHTS:")
    print(f"  • Best performing method: {best_method} with {best_mape:.2f}% MAPE")
    print(f"  • Test period covered: {len(overall_actual)} months")
    print(f"  • Training period: {len(data_train)} months")
    if 'Ensemble' in backtest_forecasts.columns:
        ensemble_bias = np.mean(backtest_forecasts['Actual'] - backtest_forecasts['Ensemble'])
        bias_direction = "under-predicts" if ensemble_bias > 0 else "over-predicts" if ensemble_bias < 0 else "is unbiased"
        print(f"  • Ensemble forecast {bias_direction} by {abs(ensemble_bias):,.0f} units on average")
    
    display(summary_df_report.head(15))
else:
    print("⚠️ Summary report generation skipped - data unavailable")

StatementMeta(, 70147d27-c9e4-4ea3-9a6b-cfa0efb49ebb, 1604, Finished, Available, Finished)


=== GENERATING BACKTESTING SUMMARY REPORT ===
Exported actuals and forecasts by method to: backtesting_actuals_vs_forecast_by_method.csv

                 OVERALL FORECASTING BACKTESTING SUMMARY

🔹 OVERALL PERFORMANCE
--------------------------------------------------
  Best Method                   : SARIMA               (MAPE: 1.63%)

🔹 BEST METHOD METRICS
--------------------------------------------------
  Mean Absolute Error           : 428,081              (SARIMA performance)
  Root Mean Square Error        : 541,647              (SARIMA performance)
  Mean Absolute Percentage Error: 1.63%                (SARIMA performance)
  R-Squared                     : -0.8852              (SARIMA performance)
  Forecast Bias                 : 215,090              (SARIMA performance)
  Bias Percentage               : 0.83%                (SARIMA performance)

🔹 DATA COVERAGE
--------------------------------------------------
  Training Period               : 2015-01 to 2023-12   (108 mon

SynapseWidget(Synapse.DataFrame, 6bf21dc9-d831-45c3-8eff-fc52770d876d)